In [5]:
Output = c("/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.3. Biomarker Distribution Analysis/Output")
cur_date = "081723"

library(readxl)
library(openxlsx)
library(tidyverse)
library(reshape2)
library(rlang)
library(PMCMRplus)

# reading in files
cytokine_df = data.frame(read_excel("Input/Processed_Cyotkine_Data_081723.xlsx"))
proteomics_5_df = data.frame(read_excel("Input/Processed_Proteomics_Data_5ug_081523.xlsx"))
proteomics_25_df = data.frame(read_excel("Input/Processed_Proteomics_Data_25ug_081523.xlsx"))
mRNA_df = data.frame(read_excel("Input/Processed_mRNA_Data_081723.xlsx"))

# making some cols numeric
cytokine_df$Condensate_Conc = as.numeric(cytokine_df$Condensate_Conc)
mRNA_df$Concentration = as.numeric(mRNA_df$Concentration)
mRNA_df$Time_Point = as.numeric(mRNA_df$Time_Point)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [6]:
head(cytokine_df)
head(proteomics_25_df)
head(mRNA_df)

,Subject_ID,Subject_No,Cytokine,Condensate,Burn_Condition,Condensate_Conc,Norm_Cytokine_Conc
,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,F_1,1,Eotaxin3,C,F,1,5.438924
2,F_1,1,Eotaxin3,C,F,25,3.630025
3,F_1,1,Eotaxin3,C,F,5,4.694272
4,F_1,1,Eotaxin3,C,S,1,6.165522
5,F_1,1,Eotaxin3,C,S,25,3.703871
6,F_1,1,Eotaxin3,C,S,5,4.562595


,Subject_No,Subject_ID,Protein_Accession,Description,Condensate,Burn_Condition,Concentration,Norm_Intensity
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,1,F_1,CO3_HUMAN,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,30.47131
2,2,M_2,CO3_HUMAN,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,31.82586
3,3,M_3,CO3_HUMAN,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,31.86083
4,4,F_4,CO3_HUMAN,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,32.32313
5,5,F_5,CO3_HUMAN,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,31.27320
6,6,M_6,CO3_HUMAN,Complement C3 OS=Homo sapiens OX=9606 GN=C3 PE=1 SV=2,C,F,25,32.42860


,Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Norm_ddCT
,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,6,M_6,HMOX1,PBS,PBS,NA,24,3.912311
2,5,F_5,HMOX1,PBS,PBS,NA,4,3.067190
3,1,F_1,HMOX1,PBS,PBS,NA,4,3.843045
4,2,M_2,HMOX1,PBS,PBS,NA,4,3.415457
5,3,M_3,HMOX1,PBS,PBS,NA,4,3.234524
6,4,F_4,HMOX1,PBS,PBS,NA,4,3.294681


# Research Question: Are there statistically significant differences in each biomarker between burn conditions within the condensate samples?

Testing for statistical differences by comparing an individual biomarker's value (ie. concentration, intensity or ddCT) between burn conditions (ie. control, smoldering, and flaming) within each condensate (ie. plastic and cardboard) and time point (ie. 4, 24, 72).

A Friedman test is a non-paramteric alternative to an ANOVA. A Nemenyi post-hoc will be subsequently run. [ref](https://www.rdocumentation.org/packages/PMCMR/versions/4.3/topics/posthoc.friedman.nemenyi.test) [ref](https://www.statology.org/friedman-test-r/)


# Friedman's Test

In [7]:
friedmans_burn_condition = function(df, concentration, value, biomarker, biomarker_name){
    # """
    # Running Friedman's tests comparing burn conditions after filtering for biomarkers, condensate, and
    # concentration. 
    # Ultimately using this test to compare biomarker expression (control, smoldering, flaming).

    # :param: dataframe, concentration variable, value, biomarker name, biomarker variable name
    # :output: a dataframe containing the biomarker, variable name, condensate, concentration, time point, 
    # comparison, stat, p value, p adj

    # """
    
    # variables that will be iterated through
    unique_biomarker = unique(df[[biomarker_name]])
    unique_condensate = c("C", "P")
    unique_conc = na.omit(unique(df[[concentration]]))
    
    friedmans_df = data.frame()
    nemenyis_df = data.frame()
    # iterating through each concentration
    for(i in 1:length(unique_biomarker)){
        for(j in 1:length(unique_condensate)){
            for(k in 1:length(unique_conc)){
                  
                # filtering for each biomarker, conc
                filtered_df = df %>%
                    filter(eval(rlang::parse_expr(biomarker_name)) == unique_biomarker[i], Condensate == "PBS" 
                           | Condensate == unique_condensate[j] &
                           eval(rlang::parse_expr(concentration)) == unique_conc[k])


                # friedman's
                friedmans_test = friedman.test(filtered_df[[value]], filtered_df$Burn_Condition, 
                                               filtered_df$Subject_ID)
                friedmans_stat_pvalue = c(friedmans_test$statistic, friedmans_test$p.value)


                # Nemenyi's post hoc
                nemenyis_test = frdAllPairsNemenyiTest(filtered_df[[value]], filtered_df$Burn_Condition, 
                                                       filtered_df$Subject_ID)
                nemenyis_stats = c(nemenyis_test$statistic[1], nemenyis_test$statistic[2], 
                                   nemenyis_test$statistic[4])
                # calculating FC to get directionality
                control_df = filtered_df %>%
                    filter(Condensate == "PBS")
                smoldering_df = filtered_df %>%
                    filter(Burn_Condition == "S")
                flaming_df = filtered_df %>%
                    filter(Burn_Condition == "F")
                
                FC = c(log2(mean(flaming_df[[value]])/mean(control_df[[value]])), # PBS vs. F
                       log2(mean(flaming_df[[value]])/mean(smoldering_df[[value]])), # S vs. F
                       log2(mean(smoldering_df[[value]])/mean(control_df[[value]]))) # PBS vs. S
                
                nemenyis_pvalues = c(nemenyis_test$p.value[1], nemenyis_test$p.value[2], 
                                    nemenyis_test$p.value[4])
                nemenyis_comparison = c(paste0(colnames(nemenyis_test$statistic)[1], " vs. ", 
                                                rownames(nemenyis_test$statistic)[1]),
                                         paste0(colnames(nemenyis_test$statistic)[1], " vs. ", 
                                                rownames(nemenyis_test$statistic)[2]),
                                         paste0(colnames(nemenyis_test$statistic)[2], " vs. ", 
                                                rownames(nemenyis_test$statistic)[2]))


                # contains biomarker, biomarker name, condensate, conc, stat, and p value
                friedmans_vector = cbind(biomarker, unique_biomarker[i], unique_condensate[j], unique_conc[k], 
                                         NA, friedmans_test$statistic, friedmans_test$p.value)
                nemenyis_matrix = cbind(biomarker, unique_biomarker[i], unique_condensate[j], unique_conc[k], 
                                        NA, nemenyis_comparison, nemenyis_stats, FC, nemenyis_pvalues)
                friedmans_df = rbind(friedmans_df, friedmans_vector)
                nemenyis_df = rbind(nemenyis_df, nemenyis_matrix)
                
            }
        }
    }
    
    # adding col names
    colnames(friedmans_df) = c("Biomarker", "Variable Name", "Condensate", "Concentration", "Time Point", 
                               "Statistic", "P Value")
    colnames(nemenyis_df) = c("Biomarker", "Variable Name", "Condensate", "Concentration", "Time Point", 
                               "Comparison", "Statistic", "FC", "P Value")
    
    # calculating padj values
    friedmans_df = friedmans_df %>%
        group_by(Condensate, Concentration) %>%
        mutate(`P Adj` = p.adjust(as.numeric(as.character(`P Value`)), method = "fdr"))
    nemenyis_df = nemenyis_df %>%
        group_by(Condensate, Concentration) %>%
        mutate(`P Adj` = p.adjust(as.numeric(as.character(`P Value`)), method = "fdr"))

    
    return(list(friedmans_df, nemenyis_df))
}

# calling fn
cytokine_friedmans = friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", 
                                              "Cytokine", "Cytokine")
proteomics_5_friedmans = friedmans_burn_condition(proteomics_5_df, "Concentration","Norm_Intensity", 
                                                  "Protein", "Protein_Accession")
proteomics_25_friedmans = friedmans_burn_condition(proteomics_25_df, "Concentration","Norm_Intensity", 
                                                   "Protein", "Protein_Accession")

Warning message in friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", :
“NaNs produced”
Warning message in friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", :
“NaNs produced”
Warning message in friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", :
“NaNs produced”
Warning message in friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", :
“NaNs produced”
Warning message in friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", :
“NaNs produced”
Warning message in friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", :
“NaNs produced”
Warning message in friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", :
“NaNs produced”
Warning message in friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", :
“NaNs produced”
Warning message in friedmans_burn_condition(cytokine_df, "Condensate_Con

Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Inte

Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Inte

Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Inte

Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_5_df, "Concentration", "Norm_Inte

...

Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "

Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "

Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "

Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "

Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "Norm_Intensity", :
“NaNs produced”
Warning message in friedmans_burn_condition(proteomics_25_df, "Concentration", "

In [8]:
# the mRNA df doesn't consistently test all 3 concentrations (1,5, or 25 micrograms) or 
# time points(4,24,72) therefore we can't subset the df using a loop so we'll create separate dfs
# for each concentration and time point
split_mRNA_df = mRNA_df %>%
    group_by(Concentration, Time_Point) %>%
    group_split()

conc1_24_mRNA_df = split_mRNA_df[[1]]
conc1_72_mRNA_df = split_mRNA_df[[2]]
conc25_4_mRNA_df = split_mRNA_df[[3]]
conc25_24_mRNA_df = split_mRNA_df[[4]]
conc25_72_mRNA_df = split_mRNA_df[[5]]
conc5_24_mRNA_df = split_mRNA_df[[6]]
conc5_72_mRNA_df = split_mRNA_df[[7]]
concc_4_mRNA_df = split_mRNA_df[[8]]
concc_24_mRNA_df = split_mRNA_df[[9]]
concc_72_mRNA_df = split_mRNA_df[[10]]

# control samples have a concentration of NA, so they're all in a separate df
# adding them back into the other dataframes
conc1_24_mRNA_df = unique(rbind(conc1_24_mRNA_df, concc_24_mRNA_df))
conc1_72_mRNA_df = unique(rbind(conc1_72_mRNA_df, concc_72_mRNA_df))
conc25_4_mRNA_df = unique(rbind(conc25_4_mRNA_df, concc_4_mRNA_df))
conc25_24_mRNA_df = unique(rbind(conc25_24_mRNA_df, concc_24_mRNA_df))
conc25_72_mRNA_df = unique(rbind(conc25_72_mRNA_df, concc_72_mRNA_df))
conc5_24_mRNA_df = unique(rbind(conc5_24_mRNA_df, concc_24_mRNA_df))
conc5_72_mRNA_df = unique(rbind(conc5_72_mRNA_df, concc_72_mRNA_df))

head(conc1_24_mRNA_df)

Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Norm_ddCT
<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
4,F_4,HMOX1,P,F,1,24,3.48011747
6,M_6,HMOX1,P,S,1,24,-0.06824032
1,F_1,HMOX1,C,F,1,24,4.63241245
1,F_1,HMOX1,P,F,1,24,1.49163928
1,F_1,HMOX1,P,S,1,24,2.47454559
2,M_2,HMOX1,C,F,1,24,2.43869489


In [9]:
friedmans_burn_condition_v2 = function(df, value, biomarker){
    # """
    # Running Friedman's tests comparing burn conditions after filtering for biomarkers and condensate. 
    # Ultimately using this test to compare biomarker expression (control, smoldering, flaming).

    # :param: dataframe, value, biomarker variable name
    # :output: a dataframe containing the biomarker, variable name, condensate, concentration, time point, 
    # comparison, stat, p value, p adj

    # """
    
    # variables that will be iterated through
    unique_biomarker = unique(df[[biomarker]])
    unique_condensate = c("C", "P")
    
    friedmans_df = data.frame()
    nemenyis_df = data.frame()
    # iterating through each concentration
    for(i in 1:length(unique_biomarker)){
        for(j in 1:length(unique_condensate)){
                  
            # filtering for each biomarker, conc
            filtered_df = df %>%
                filter(eval(rlang::parse_expr(biomarker)) == unique_biomarker[i], 
                       Condensate != unique_condensate[j])


            # friedman's
            friedmans_test = friedman.test(filtered_df[[value]], filtered_df$Burn_Condition, 
                                           filtered_df$Subject_ID)
            friedmans_stat_pvalue = c(friedmans_test$statistic, friedmans_test$p.value)


            # Nemenyi's post hoc
            nemenyis_test = frdAllPairsNemenyiTest(filtered_df[[value]], filtered_df$Burn_Condition, 
                                                   filtered_df$Subject_ID)
            nemenyis_stats = c(nemenyis_test$statistic[1], nemenyis_test$statistic[2], 
                               nemenyis_test$statistic[4])
           # calculating FC to get directionality
            control_df = filtered_df %>%
                filter(Condensate == "PBS")
            smoldering_df = filtered_df %>%
                filter(Burn_Condition == "S")
            flaming_df = filtered_df %>%
                filter(Burn_Condition == "F")
                
            FC = c(log2(mean(flaming_df[[value]])/mean(control_df[[value]])), # PBS vs. F
                   log2(mean(flaming_df[[value]])/mean(smoldering_df[[value]])), # S vs. F
                   log2(mean(smoldering_df[[value]])/mean(control_df[[value]]))) # PBS vs. S
            
            nemenyis_pvalues = c(nemenyis_test$p.value[1], nemenyis_test$p.value[2], 
                                nemenyis_test$p.value[4])
            nemenyis_comparison = c(paste0(colnames(nemenyis_test$statistic)[1], " vs. ", 
                                            rownames(nemenyis_test$statistic)[1]),
                                     paste0(colnames(nemenyis_test$statistic)[1], " vs. ", 
                                            rownames(nemenyis_test$statistic)[2]),
                                     paste0(colnames(nemenyis_test$statistic)[2], " vs. ", 
                                            rownames(nemenyis_test$statistic)[2]))


            # contains biomarker, biomarker name, conc, time point, condensate, stat, and p value
            friedmans_vector = cbind(biomarker, unique_biomarker[i], unique_condensate[j], 
                                     unique(filtered_df$Concentration)[1], unique(filtered_df$Time_Point),
                                     friedmans_test$statistic, friedmans_test$p.value)
            nemenyis_matrix = cbind(biomarker, unique_biomarker[i], unique_condensate[j],
                                    unique(filtered_df$Concentration)[1], unique(filtered_df$Time_Point),
                                     nemenyis_comparison, nemenyis_stats, FC, nemenyis_pvalues)
            friedmans_df = rbind(friedmans_df, friedmans_vector)
            nemenyis_df = rbind(nemenyis_df, nemenyis_matrix)

        }
    }
    
    # adding col names
    colnames(friedmans_df) = c("Biomarker", "Variable Name", "Condensate", "Concentration", "Time Point", 
                               "Statistic", "P Value")
    colnames(nemenyis_df) = c("Biomarker", "Variable Name", "Condensate", "Concentration", "Time Point", 
                               "Comparison", "Statistic", "FC", "P Value")
    
    # calculating padj values
    friedmans_df = friedmans_df %>%
        group_by(Condensate) %>%
        mutate(`P Adj` = p.adjust(as.numeric(as.character(`P Value`)), method = "fdr"))
    nemenyis_df = nemenyis_df %>%
        group_by(Condensate) %>%
        mutate(`P Adj` = p.adjust(as.numeric(as.character(`P Value`)), method = "fdr"))

    
    return(list(friedmans_df, nemenyis_df))
}

# calling fn

conc1_24_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc1_24_mRNA_df, "Norm_ddCT", "mRNA")
conc1_72_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc1_72_mRNA_df, "Norm_ddCT", "mRNA")
conc25_4_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc25_4_mRNA_df, "Norm_ddCT", "mRNA")
conc25_24_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc25_24_mRNA_df, "Norm_ddCT", "mRNA")
conc25_72_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc25_72_mRNA_df, "Norm_ddCT", "mRNA")
conc5_24_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc5_24_mRNA_df, "Norm_ddCT", "mRNA")
conc5_72_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc5_72_mRNA_df, "Norm_ddCT", "mRNA")

Warning message in friedmans_burn_condition_v2(conc1_24_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_24_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_24_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_24_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_24_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_24_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_24_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_24_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_24_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_24_mRNA_df, "Norm_dd

Warning message in friedmans_burn_condition_v2(conc1_72_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_72_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_72_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_72_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_72_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_72_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_72_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_72_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_72_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_72_mRNA_df, "Norm_dd

ERROR: Error in friedman.test.default(filtered_df[[value]], filtered_df$Burn_Condition, : not an unreplicated complete block design


In [10]:
# creating 1 df
friedmans_df = rbind(cytokine_friedmans[[1]], proteomics_25_friedmans[[1]], proteomics_5_friedmans[[1]],
                     conc1_24_friedmans_mRNA_v2[[1]], conc1_72_friedmans_mRNA_v2[[1]], 
                     conc25_4_friedmans_mRNA_v2[[1]], conc25_24_friedmans_mRNA_v2[[1]], 
                     conc25_72_friedmans_mRNA_v2[[1]], conc5_24_friedmans_mRNA_v2[[1]],
                     conc5_72_friedmans_mRNA_v2[[1]])
nemenyis_df = rbind(cytokine_friedmans[[2]], proteomics_25_friedmans[[2]], proteomics_5_friedmans[[2]],
                    conc1_24_friedmans_mRNA_v2[[2]], conc1_72_friedmans_mRNA_v2[[2]], 
                    conc25_4_friedmans_mRNA_v2[[2]], conc25_24_friedmans_mRNA_v2[[2]], 
                    conc25_72_friedmans_mRNA_v2[[2]], conc5_24_friedmans_mRNA_v2[[2]],
                    conc5_72_friedmans_mRNA_v2[[2]])

head(friedmans_df)
head(nemenyis_df)

ERROR: Error in list2(...): object 'conc25_4_friedmans_mRNA_v2' not found


In [7]:
# exporting
write.xlsx(friedmans_df, paste0(Output,"/", "Friedmans_Biomarker_Results_", cur_date, ".xlsx"), 
           rowNames = FALSE)
write.xlsx(nemenyis_df, paste0(Output,"/", "Nemenyis_PostHoc_Biomarker_Results_", cur_date, ".xlsx"), 
           rowNames = FALSE)